# Sentence BERT

* [Sentence Transformers Documentation](https://www.sbert.net/)
* 参考サイト
* https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part18.html

<a href="https://colab.research.google.com/github/fuyu-quant/Data_Science/blob/main/Natural_Language_processing/Text_Embedding(english)/SenteceBERT_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers

In [3]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import InputExample
from sentence_transformers import losses
from torch.utils.data import DataLoader


from sentence_transformers.evaluation import ParaphraseMiningEvaluator

from sentence_transformers.datasets import ParallelSentencesDataset

import math

## データの用意

In [6]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

## SentenceTransformer
* 事前学習モデル  
https://www.sbert.net/docs/pretrained_models.html#sentence-embedding-models/

In [7]:
model = SentenceTransformer('all-mpnet-base-v2')
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(sentences)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

## Embeddingの出力

In [10]:
embeddings

array([[ 0.00641698,  0.00704137, -0.02814424, ...,  0.04231948,
        -0.09057346, -0.05940839],
       [ 0.07764477, -0.05905227, -0.03911835, ..., -0.00052688,
        -0.05023519, -0.02451303],
       [-0.03429268, -0.00133944,  0.00433614, ...,  0.01533877,
         0.09794599,  0.01603547]], dtype=float32)

## ファインチューニングの方法

In [11]:
# パラメータ
train_batch_size = 16
max_seq_length = 128
num_epochs = 5

In [12]:
# 文章のペアとその類似度が学習データ
train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
    InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]


# データローダー
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size = train_batch_size)

In [ ]:
# 損失関数の定義
train_loss = losses.MultipleNegativesRankingLoss(model)
#train_loss = losses.CosineSimilarityLoss(model)


#dev_evaluator = ParaphraseMiningEvaluator(sentences_map, duplicates_list, name="paramin-jsnli-dev")




# ファインチューニング
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs = num_epochs, warmup_steps=100)
model.fit(train_objectives=[(train_dataloader, train_loss)], 
          #evaluator=dev_evaluator,
          epochs = num_epochs, 
          evaluation_steps=int(len(train_dataloader)*0.1),
          #warmup_steps = warmup_steps
          )